In [16]:
import numpy as np
import cv2
from tqdm import tqdm

In [17]:
cap = cv2.VideoCapture('video1.mp4')

n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('video_out.mp4', fourcc, fps, (w, h))

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [18]:
cv2.__version__

'4.5.3'

In [19]:
# read first frame and convert it to grayscale
_, prev = cap.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

# pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32)

In [20]:
for i in tqdm(range(n_frames - 2)):
    # detect feature points in previous frame
    prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                      maxCorners = 200,
                                      qualityLevel = 0.01,
                                      minDistance = 30,
                                      blockSize = 3)
    
    # read next frame
    success, curr = cap.read()
    if not success:
        break
        
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,
                                                    curr_gray,
                                                    prev_pts,
                                                    None)
    
    #sanity check
    assert prev_pts.shape == curr_pts.shape
    
    # filter only valid points
    idx = np.where(status == 1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]
    
    # find transformation matrix
    m, _ = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
    
    # extract translation
    dx = m[0,2]
    dy = m[1,2]
    
    # extract rotation angle
    da = np.arctan2(m[1,0], m[0,0])
    
    # store transformation
    transforms[i] = [dx, dy, da]
    
    # move to next frame
    prev_gray = curr_gray
    
    #print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

100%|██████████████████████████████████████| 1284/1284 [00:06<00:00, 198.63it/s]


In [21]:
m

array([[ 9.99644513e-01,  1.52596116e-04, -2.90327380e-01],
       [-1.52596116e-04,  9.99644513e-01, -1.23836176e-01]])

In [22]:
trajectory = np.cumsum(transforms, axis = 0)

In [23]:
def movingAverage(curve, radius):
    window_size = 2 * radius + 1
    f = np.ones(window_size) / window_size
    curve_pad = np.pad(curve, (radius, radius), 'edge')
    curve_smoothed = np.convolve(curve_pad, f, mode = 'same')
    curve_smoothed = curve_smoothed[radius:-radius]
    return curve_smoothed

In [24]:
def smooth(trajectory, radius):
    smoothed_trajectory = np.copy(trajectory)
    for i in range(3):
        smoothed_trajectory[:,i] = movingAverage(trajectory[:,i],
                                                radius = radius)
        return smoothed_trajectory

In [25]:
smoothed_trajectory = smooth(trajectory, 100)

In [26]:
# calculate difference in smoothed trajectory and trajectory
difference = smoothed_trajectory - trajectory

# calculate newer transformation array
transforms_smooth = transforms + difference

In [27]:
def fixBorder(frame):
    s = frame.shape
    # scale the image without moveng the center
    T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
    frame = cv2.warpAffine(frame, T, (s[1], s[0]))
    return frame

In [28]:
# reset stream to first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

if(cap.isOpened() == False):
    print('Error opening video stream')
    
i = 0
while(cap.isOpened()):
    if i >= n_frames-1:
        break
    ret, frame = cap.read()
    
    if ret == True:
        # extract transformations from the new transformation array
        dx = transforms_smooth[i,0]
        dy = transforms_smooth[i,1]
        da = transforms_smooth[i,2]

        # reconstruct transformation matrix accordingly to new values
        m = np.zeros((2,3), np.float32)
        m[0,0] = np.cos(da)
        m[0,1] = -np.sin(da)
        m[1,0] = np.sin(da)
        m[1,1] = np.cos(da)
        m[0,2] = dx
        m[1,2] = dy

        # apply affine wrapping to the given frame
        frame_stabilized = cv2.warpAffine(frame, m, (w,h))

        # fix border artifacts
        frame_stabilized = fixBorder(frame_stabilized)

        # write the frame to the file
        frame_out = cv2.hconcat([frame, frame_stabilized])

        # if the image is too big, resize it
        if(frame_out.shape[1] > 1920):
            frame_out = cv2.resize(frame_out, 
                                   (frame_out.shape[1] / 2,
                                   frame_out.shape[0] / 2))
        
        cv2.imshow('Before and after', frame_out)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        i += 1       
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to target thread (0x7f8690177d80)

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to target thread (0x7f8690177d80)

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to target thread (0x7f8690177d80)

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to target thread (0x7f8690177d80)

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to target thread (0x7f8690177d80)

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to target thread (0x7f8690177d80)

QObject::moveToThread: Current thread (0x7f8690177d80) is not the object's thread (0x5639c0fe3610).
Cannot move to tar